In [65]:
import pandas as pd
import geopandas as gpd
import dask_geopandas as dgpd
from shapely.geometry import shape
import shapely
from shapely import wkt
import dask.dataframe as dd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import Stamen

In [29]:
rise_df = pd.read_csv('../data/US-RISE.csv', parse_dates=['time'])
rise_df

,Unnamed: 0,time,values,location,product,geometry,catalog_item,catalog_record
0,9,1970-01-01 19:00:00+00:00,1.300,Edward Arthur Patterson Lake,Lake/Reservoir Release - Total,POINT (-102.8269 46.8697),308,2288
1,10,1970-01-02 19:00:00+00:00,1.400,Edward Arthur Patterson Lake,Lake/Reservoir Release - Total,POINT (-102.8269 46.8697),308,2288
2,11,1970-01-03 19:00:00+00:00,1.400,Edward Arthur Patterson Lake,Lake/Reservoir Release - Total,POINT (-102.8269 46.8697),308,2288
3,12,1970-01-04 19:00:00+00:00,1.300,Edward Arthur Patterson Lake,Lake/Reservoir Release - Total,POINT (-102.8269 46.8697),308,2288
4,13,1970-01-05 19:00:00+00:00,1.400,Edward Arthur Patterson Lake,Lake/Reservoir Release - Total,POINT (-102.8269 46.8697),308,2288
...,...,...,...,...,...,...,...,...
15785062,19225,2022-10-26 06:00:00+00:00,203.863,Sun River Diversion Dam,Streamflow,POINT (-112.7067 47.6208),762,2451
15785063,19226,2022-10-27 06:00:00+00:00,202.177,Sun River Diversion Dam,Streamflow,POINT (-112.7067 47.6208),762,2451
15785064,19227,2022-10-28 06:00:00+00:00,202.994,Sun River Diversion Dam,Streamflow,POINT (-112.7067 47.6208),762,2451
15785065,19228,2022-10-29 06:00:00+00:00,202.607,Sun River Diversion Dam,Streamflow,POINT (-112.7067 47.6208),762,2451


In [76]:
# get the unique geometries
locations = []
names = []
products = []
for location, group in rise_df.groupby('geometry'):
    name = group['location'].iloc[0]
    for p, g in group.groupby('product'):
        if 'Lake/Reservoir ' in p:
            p = p.replace('Lake/Reservoir', '')
        
        locations.append(wkt.loads(location))
        products.append(p)
        names.append(name)

In [77]:
RISE_locations = gpd.GeoDataFrame({
    'name': names,
    'product': products,
    'geometry': locations
}, crs='epsg:4326')
# RISE_locations[RISE_locations['geometry']]
RISE_locations = RISE_locations[RISE_locations['geometry'].apply(lambda x: isinstance(x, shapely.geometry.Point))]

In [85]:
RISE_locations.to_csv('../data/US-RISE-metadata.csv')

In [83]:
RISE_locations.explore(     
    # column="name", # make choropleth based on "BoroName" column
    tooltip="name", # show "BoroName" value in tooltip (on hover)
    popup=True, # show all values in popup (on click)
    tiles="CartoDB positron", # use "CartoDB positron" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black") # use black outline
)